In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Dropout, Add, Conv2DTranspose
from tensorflow.keras.models import Model
from glob import glob
from tqdm import tqdm  # Progress bar


In [2]:
# ✅ Set Fixed Image Size (All images will be resized to this)
FIXED_SIZE = (256, 256)  # Change this if needed

# Directories
INPUT_DIRS = {
    "real": "E:/FAST/mid 2 eval/enhanced/real",
    "fake": "E:/FAST/mid 2 eval/enhanced/fake"
}
OUTPUT_DIR = "E:/FAST/mid 2 eval/denoised_enhanced_ONEMOREYTST"

# Ensure output directories exist
os.makedirs(os.path.join(OUTPUT_DIR, "real"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "fake"), exist_ok=True)

# ✅ Custom SSIM + MSE Loss
def ssim_loss(y_true, y_pred):
    return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, max_val=1.0)) + tf.keras.losses.MeanSquaredError()(y_true, y_pred)

# ✅ Improved Autoencoder Model (Fixed Skip Connection Shapes)
def build_denoising_autoencoder():
    input_img = Input(shape=(FIXED_SIZE[0], FIXED_SIZE[1], 3))  # Set fixed size

    # Encoder
    x1 = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x1 = BatchNormalization()(x1)
    x1 = MaxPooling2D((2, 2), padding='same')(x1)  # (128, 128, 32)

    x2 = Conv2D(64, (3, 3), activation='relu', padding='same')(x1)
    x2 = BatchNormalization()(x2)
    x2 = MaxPooling2D((2, 2), padding='same')(x2)  # (64, 64, 64)

    x3 = Conv2D(128, (3, 3), activation='relu', padding='same')(x2)
    x3 = BatchNormalization()(x3)
    x3 = MaxPooling2D((2, 2), padding='same')(x3)  # (32, 32, 128)

    # Bottleneck
    x4 = Conv2D(256, (3, 3), activation='relu', padding='same')(x3)
    x4 = BatchNormalization()(x4)
    x4 = Dropout(0.2)(x4)  # (32, 32, 256)

    # Decoder
    x5 = UpSampling2D((2, 2))(x4)  # (64, 64, 256)
    x5 = Conv2D(128, (3, 3), activation='relu', padding='same')(x5)
    x5 = BatchNormalization()(x5)

    # ✅ Resize x3 before skip connection
    x3_resized = Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same')(x3)  # (64, 64, 128)
    x5 = Add()([x5, x3_resized])  # Now both shapes match

    x6 = UpSampling2D((2, 2))(x5)  # (128, 128, 128)
    x6 = Conv2D(64, (3, 3), activation='relu', padding='same')(x6)
    x6 = BatchNormalization()(x6)

    # ✅ Resize x2 before skip connection
    x2_resized = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same')(x2)  # (128, 128, 64)
    x6 = Add()([x6, x2_resized])  # Now both shapes match

    x7 = UpSampling2D((2, 2))(x6)  # (256, 256, 64)
    x7 = Conv2D(32, (3, 3), activation='relu', padding='same')(x7)
    x7 = BatchNormalization()(x7)

    # ✅ Resize x1 before skip connection
    x1_resized = Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same')(x1)  # (256, 256, 32)
    x7 = Add()([x7, x1_resized])  # Now both shapes match

    decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x7)

    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=ssim_loss)

    return autoencoder

# ✅ Load & Resize Images
def load_images_from_folder(folder):
    image_paths = glob(os.path.join(folder, "*.*"))  # Load all images
    images = []

    for img_path in image_paths:
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # ✅ Resize all images to FIXED_SIZE
        img = cv2.resize(img, FIXED_SIZE)
        img = img / 255.0  # Normalize

        images.append(img)

    return np.array(images, dtype=np.float32), image_paths

# ✅ Denoise and Save Images
def denoise_and_save(autoencoder, category):
    print(f"\nProcessing {category} images...")
    input_folder = INPUT_DIRS[category]
    output_folder = os.path.join(OUTPUT_DIR, category)

    images, image_paths = load_images_from_folder(input_folder)

    # Predict denoised images
    denoised_images = autoencoder.predict(images)

    for img, path in tqdm(zip(denoised_images, image_paths), total=len(image_paths), desc=f"Denoising {category}"):
        img = (img * 255).astype(np.uint8)  # Convert back to uint8 format
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)  # Convert back to BGR for OpenCV

        filename = os.path.basename(path)  # Keep original filename
        cv2.imwrite(os.path.join(output_folder, filename), img)  # Save denoised image

# ✅ Train Autoencoder
def train_autoencoder():
    print("\nLoading dataset for training...")
    real_images, _ = load_images_from_folder(INPUT_DIRS["real"])
    fake_images, _ = load_images_from_folder(INPUT_DIRS["fake"])

    # Combine datasets
    all_images = np.vstack([real_images, fake_images])

    autoencoder = build_denoising_autoencoder()

    print("\nTraining denoising autoencoder...")
    autoencoder.fit(all_images, all_images, epochs=10, batch_size=8, shuffle=True)

    autoencoder.save("saved_autoencoder_model.h5")
    print("\n✅ Model saved successfully as 'saved_autoencoder_model.h5'")

    return autoencoder

# ✅ Main function
if __name__ == "__main__":
    autoencoder = train_autoencoder()

    # Process and save denoised images
    for category in ["real", "fake"]:
        denoise_and_save(autoencoder, category)

    print("\n✅ Denoising complete! All images saved with fixed size.")



Loading dataset for training...

Training denoising autoencoder...
Epoch 1/10
977/977 [==============================] - 81s 72ms/step - loss: 0.0821
Epoch 2/10
977/977 [==============================] - 80s 82ms/step - loss: 0.0506
Epoch 3/10
977/977 [==============================] - 81s 83ms/step - loss: 0.0426
Epoch 4/10
977/977 [==============================] - 80s 82ms/step - loss: 0.0342
Epoch 5/10
977/977 [==============================] - 80s 81ms/step - loss: 0.0307
Epoch 6/10
977/977 [==============================] - 79s 81ms/step - loss: 0.0299
Epoch 7/10
977/977 [==============================] - 78s 80ms/step - loss: 0.0270
Epoch 8/10
977/977 [==============================] - 77s 79ms/step - loss: 0.0259
Epoch 9/10
977/977 [==============================] - 77s 79ms/step - loss: 0.0243
Epoch 10/10
977/977 [==============================] - 76s 78ms/step - loss: 0.0234

✅ Model saved successfully as 'saved_autoencoder_model.h5'

Processing real images...


InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from glob import glob
from tqdm import tqdm  # Progress bar

# ✅ Set the same fixed size used for training
FIXED_SIZE = (256, 256)  

# ✅ Model Path (Update if needed)
MODEL_PATH = "saved_autoencoder_model.h5"

# Directories
INPUT_DIRS = {
    "real": "E:/FAST/mid 2 eval/enhanced/real",
    "fake": "E:/FAST/mid 2 eval/enhanced/fake"
}
OUTPUT_DIR = "E:/FAST/mid 2 eval/generated_enhanced"

# Ensure output directories exist
os.makedirs(os.path.join(OUTPUT_DIR, "real"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "fake"), exist_ok=True)

# ✅ Load the saved model
def load_model():
    print("\nLoading the saved model...")
    model = tf.keras.models.load_model(MODEL_PATH, compile=False)
    print("✅ Model loaded successfully!")
    return model

# ✅ Load & Resize Images
def load_images_from_folder(folder):
    image_paths = glob(os.path.join(folder, "*.*"))  # Load all images
    images = []

    for img_path in image_paths:
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Resize to fixed size (Same as training)
        img = cv2.resize(img, FIXED_SIZE)
        img = img / 255.0  # Normalize
        images.append(img)

    return np.array(images, dtype=np.float32), image_paths

# ✅ Denoise and Save Images (Using the Saved Model)
def generate_images(model, category, batch_size=16):
    print(f"\nProcessing {category} images...")
    input_folder = INPUT_DIRS[category]
    output_folder = os.path.join(OUTPUT_DIR, category)

    images, image_paths = load_images_from_folder(input_folder)
    denoised_images = []

    # Process images in batches
    for i in tqdm(range(0, len(images), batch_size), desc=f"Generating {category}"):
        batch_images = images[i:i+batch_size]  # Take a batch of images
        batch_denoised = model.predict(batch_images)  # Denoise the batch
        denoised_images.extend(batch_denoised)

    # Save all generated images
    for img, path in zip(denoised_images, image_paths):
        img = (img * 255).astype(np.uint8)  # Convert back to uint8 format
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)  # Convert back to BGR for OpenCV

        filename = os.path.basename(path)  # Keep original filename
        cv2.imwrite(os.path.join(output_folder, filename), img)  # Save generated image

# ✅ Main Function
if __name__ == "__main__":
    # Load the trained model
    autoencoder_model = load_model()

    # Generate images for both categories
    for category in ["real", "fake"]:
        generate_images(autoencoder_model, category)

    print("\n✅ Image generation complete! All generated images are saved.")



Loading the saved model...
✅ Model loaded successfully!

Processing real images...


Generating real:   0%|          | 0/151 [00:00<?, ?it/s]

1/1 [==============================] - 5s 5s/step


Generating real:   1%|          | 1/151 [00:04<11:48,  4.73s/it]

1/1 [==============================] - 0s 17ms/step


...

1/1 [==============================] - 0s 16ms/step


Generating fake:  99%|█████████▉| 337/339 [00:26<00:00, 12.49it/s]

1/1 [==============================] - 0s 337ms/step


Generating fake: 100%|██████████| 339/339 [00:26<00:00, 12.59it/s]



✅ Image generation complete! All generated images are saved.
